# 🎬 Movie Recommendation System Demo

This notebook demonstrates how to use the content-based movie recommender system.

In [ ]:
!pip install -r ../requirements.txt

## 📥 Load the Recommender

In [ ]:
from src.recommender import MovieRecommendationSystem

movies_path = '../data/tmdb_5000_movies.csv'
credits_path = '../data/tmdb_5000_credits.csv'
recommender = MovieRecommendationSystem(movies_path, credits_path)
recommender.load_and_preprocess_data()
recommender.build_similarity_matrix()

## 🎯 Get Recommendations

In [ ]:
movie = 'Avatar'
results = recommender.get_recommendations(movie)

print(f"\n🎬 Recommendations for '{movie}':")
for i, rec in enumerate(results['recommendations'], 1):
    print(f"{i}. {rec['title']} (Rating: {rec['rating']}, Similarity: {rec['similarity_score']})")